In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'kitchen_&_housewares'

hNeg = False #if true, add negative bigrams for negative reviews
noun = True #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 6318 palavras!


In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [3]:
# Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
# Xval = Xval[:, index]

#Converte matrizes esparsas para np arrays, para os cálculos da regressão logística
Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

In [4]:
# print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
# print("----------------------------------------")
# print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
#print(new_vocabulary)

In [5]:
def sigmoid(z):
    
    """
    Calcula a funcao sigmoidal  
    """
    
    if isinstance(z, int):
        g = 0
    
    # se z não é um inteiro, significa que é um array e inicia com a dimensão do array
    else:
        g = np.zeros( z.shape );

    g = 1/(1 + np.exp(-z))
  
    return g

In [6]:
def funcaoCustoReg(theta, X, Y, lambda_reg):
    
    """
    Calcula o custo da regressao logística
    """
    
    hipotese = 0
    reg = 0
    
    hipotese = sigmoid(np.dot(X, theta))
    
    cost = (-Y * np.log(hipotese + eps)) - ((1 - Y) * np.log(1 - hipotese + eps))
    
    reg = (lambda_reg/(2*m) * np.sum(theta[1:] ** 2))
         
    J = (1/m * np.sum(cost)) + reg
        
    grad[0] = (np.dot(X.T, hipotese - Y)[0])/m
    grad[1:] = (np.dot(X.T, hipotese - Y)[1:])/m + (lambda_reg/m)*theta[1:]
                      
    return J, grad


In [7]:
def gridSearch(X, Y ,Xval, Yval, iteracoes):
    
    lambda_reg = [0.01, 0.05, 0.1, 0.5, 10, 50, 100, 500]

    bestLambda = lambda_reg[0]
    
    bestAcc = 0

    for l in lambda_reg :
        theta = treinamento(X,Y, l, iteracoes)
        Ypred, acuracia = predicao(Xval, theta, Yval)
        if bestAcc < acuracia :
            bestLambda = l
            bestAcc = acuracia


    return bestLambda


In [8]:
import scipy
import scipy.optimize 

def treinamento(X, Y, lambda_reg, iteracoes):
        
    # se for vazio, retorna None 
    if len(Y)==0:
        return None
    
    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    theta = np.zeros(n) # Inicializa parâmetros que serao ajustados
    # minimiza a funcao de custo
    result = scipy.optimize.minimize(fun=funcaoCustoReg, x0=theta, args=(X, Y, lambda_reg),  
                method='BFGS', jac=True, options={'maxiter': iteracoes, 'disp':False})

    # coleta os thetas retornados pela função de minimização
    theta = result.x
    
    return theta

In [ ]:
def predicao(X, theta, Yval):
    
    """
    Prediz se a entrada pertence a classe 0 ou 1 usando o parametro
    theta obtido pela regressao logistica
   
    """
    m = X.shape[0]   
    
    p = np.zeros(m, dtype=int) 
    z = np.dot(X, theta)
    h_theta = sigmoid(z) 
    
    for i in range(m):
        if h_theta[i] >= 0.5:
            p[i] = 1 
            
    acuracia = np.sum(p==Yval)/len(Yval)
    
    return p, acuracia

In [ ]:
numIter = 50
lambdaReg = gridSearch(Xtrain, Ytrain, Xval, Yval, numIter)
theta = treinamento(Xtrain, Ytrain, lambdaReg, numIter)

In [ ]:
Ypred, acuracia = predicao(Xval, theta, Yval)
print("Acurácia é "+ str(acuracia))

In [ ]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

In [ ]:
def curva_aprendizado(Xtrain, Ytrain, Xval, Yval, lambdaReg, numIter):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        theta = treinamento(Xtrain[:i], Ytrain[:i], numIter, lambdaReg)
        
        Ypred, acuracia = predicao(Xtrain[:i], theta, Ytrain[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Xval, theta, Yval)
        perf_val.append(acuracia)

    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()



In [ ]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval, lambdaReg, numIter)

In [ ]:
import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold

for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    Xtrain, Xval = X3[train_index, :], X3[test_index, :];
    Ytrain, Yval= Y3[train_index], Y3[test_index];

#     Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
#     Xval = Xval[:, index]
    
    #Converte matrizes esparsas para np arrays, para os cálculos da regressão logística
    Xtrain = Xtrain.toarray()
    Xval = Xval.toarray()

    numIter = 50
    
    lambdaReg = gridSearch(Xtrain, Ytrain, Xval, Yval, numIter)
    
    print("Melhor lambda")
    print(lambdaReg)
    
    theta = treinamento(Xtrain, Ytrain, lambdaReg, numIter)
    
    Ypred, acuracia = predicao(Xval, theta, Yval)

    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults )
    
    k = k + 1
    
kf.mediaFolds( resultados, classes )